# Prepare Environment

In [6]:
import numpy as np
import pandas as pd
import nltk

nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')
nltk.download('stopwords')



[nltk_data] Downloading package punkt to
[nltk_data]     /Users/kristian.aars/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/kristian.aars/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kristian.aars/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kristian.aars/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
!pip install autocorrect tensorflow numpy keras regex pyyaml h5py contractions pandarallel


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Download blogtext.csv (700 000 blog posts)
!gdown 1PJbVYUmRr0_HTwGNtplnu8lG-UCDoXZJ

# Download blogtext_cleaned.csv (10 000 blog posts, 43 500 sentences)
!gdown 1qI-TZrQ_D0S0g7O3l_qKKA7f4l70jlmf

# Load Data

In [47]:
%%time
# Import dataset from CSV

df = pd.read_csv('blogtext.csv').head(5000)

CPU times: user 5.74 s, sys: 608 ms, total: 6.35 s
Wall time: 6.66 s


In [48]:
print(df.shape)
df.head(10)

(5000, 7)


,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Take a read of this news article ...
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004",I surf the English news sites a l...


# Tokenize sentences

In [49]:
%%time
df.text = df.text.transform(lambda t: nltk.sent_tokenize(t))
df.shape

CPU times: user 412 ms, sys: 7.76 ms, total: 419 ms
Wall time: 423 ms


(5000, 7)

In [51]:
df = df.explode('text')

print(df.shape)

mask = df['text'].apply(lambda x: isinstance(x, str))
df = df[mask]

df.shape

(50918, 7)


(50918, 7)

In [52]:
#df.text.to_csv("blogtext-sentence_tokenized.csv")

# Prepearing data for training

In [53]:
from keras.src.preprocessing.text import Tokenizer
from keras.src.utils import pad_sequences, to_categorical
import re
import random
from pandarallel import pandarallel

import numpy as np
from nltk import ngrams
from nltk.corpus import words as en_words
from nltk.corpus import stopwords

import spacy
from autocorrect import Speller

spacy_nlp = spacy.load("en_core_web_sm")

In [54]:
from nltk import word_tokenize

def autocorrect_corpus(corpus):
    speller = Speller(lang='en')
    return corpus.transform(lambda s: speller(s['text']))

def has_non_lexi_word(sentence):
    sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
    lemmatized_doc = spacy_nlp(sentence)
    
    english_words = en_words.words()
    
    for token in lemmatized_doc:
        word = token.lemma_.lower()
        
        if word not in english_words:                               
            #print("Found non-english word {0}".format(word))
            return True

def contains_number(sentence):
    return any(char.isdigit() for char in sentence)

def remove_stop_words(text):
    # Tokenize the input text into words
    words = word_tokenize(text)
    
    # Get the list of English stop words
    stop_words = set(stopwords.words('english'))
    
    # Remove stop words from the list of words
    filtered_words = [word for word in words if word.lower() not in stop_words]
    
    # Join the filtered words back into a string
    filtered_text = ' '.join(filtered_words)
    
    return filtered_text

def word_count(text):
    words = word_tokenize(text)
    return len(words)

def clean_corpus(corpus_df, rm_non_lexi_word_sentence=True, rm_contains_num=True, min_word_count=2, remove_stopwords=True, prob_remove_stopword=0.1):
    
    def remove_sentences_condition(row):
        if word_count(row['text']) < min_word_count: return False
        elif rm_contains_num and contains_number(row['text']): return False
        elif rm_non_lexi_word_sentence and has_non_lexi_word(row['text']): return False
        else: return True
    
    # Function to normalize and clean the text
    def clean_text(text):
        # Convert to lowercase for better normalization
        text = text.lower()
    
        # Remove punctuation
        text = re.sub(r'[^\w\s]', '', text)
    
        # Tokenize the words
        words = word_tokenize(text)
        
        # Remove stop words
        if remove_stopwords:
            if random.random() < prob_remove_stopword:
                stop_words = set(stopwords.words('english'))
                words = [word for word in words if word not in stop_words]
                
        # Join the cleaned words back into a sentence
        cleaned_text = ' '.join(words)
    
        return cleaned_text
    
    pre_rem_size = corpus_df.shape[0]
    pandarallel.initialize()
    corpus_df['text'] = corpus_df['text'].apply(clean_text)
    corpus_df = corpus_df[corpus_df.parallel_apply(remove_sentences_condition, axis=1)]
    sen_removed = pre_rem_size - corpus_df.shape[0]
    print("Removed {0} sentences because they contained email, phone, or url(s)".format(sen_removed))

    # Run autocorrect to fix text-typos
    # autocorrect_corpus(corpus_df)

    # En løsning er å fjerne alle ord som ikke eksisterer i det engelske vokabularet.

    return corpus_df


In [55]:
df = clean_corpus(df, rm_sentence_phone=False, rm_sentence_url=False, rm_sentence_email=False, remove_stopwords=False, rm_non_lexi_word_sentence=True, prob_remove_stopword=0.10)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Removed 29555 sentences because they contained email, phone, or url(s)


In [56]:
df

# Remove sentences with less than two words


,id,gender,age,topic,sign,date,text
2,2059027,male,15,Student,Leo,"12,May,2004",seemed to be a transcript of a seven days article
2,2059027,male,15,Student,Leo,"12,May,2004",poorly formatted and corrupted
2,2059027,male,15,Student,Leo,"12,May,2004",i have added the text between examine under a ...
2,2059027,male,15,Student,Leo,"12,May,2004",if anyone has the full text please distribute
2,2059027,male,15,Student,Leo,"12,May,2004",i am not responsible for the accuracy of this ...
...,...,...,...,...,...,...,...
4995,1103575,female,17,indUnk,Scorpio,"27,August,2003",and everybody just sort of looked at each othe...
4996,1103575,female,17,indUnk,Scorpio,"27,August,2003",tomorrow i think i will be depressed
4996,1103575,female,17,indUnk,Scorpio,"27,August,2003",i think ill go now
4999,1103575,female,17,indUnk,Scorpio,"26,August,2003",when i got home i did laundry


In [57]:
df.to_csv("blogtext_cleaned.csv")

In [15]:
import pandas as pd

df = pd.read_csv('blogtext_cleanedv2.csv')

#df['text'] = df['text'].apply(remove_stop_words)

df = df['text'].head(10000)

df.shape

(10000,)

In [16]:
df

0       seemed to be a transcript of a seven days article
1                          poorly formatted and corrupted
2       i have added the text between examine under a ...
3           if anyone has the full text please distribute
4       i am not responsible for the accuracy of this ...
                              ...                        
9995    and this is not to say that one way is better ...
9996    we make lots more money here because of our pr...
9997        its part of what makes us a capitalist nation
9998    but its also a big part of why were a fat nati...
9999    i bristle when it is suggested that in the who...
Name: text, Length: 10000, dtype: object

In [17]:
sentence_list = df.tolist()

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentence_list)
total_words = len(tokenizer.word_index) + 1

NameError: name 'Tokenizer' is not defined

In [ ]:
import matplotlib.pyplot as plt


c_dict = {}

for s in df:
    print(s)
    wrds = s.split()
    for w in wrds:
        if w in c_dict.keys():
            c_dict[w] += 1
        else:
            c_dict[w] = 1
            
c_dict = dict(sorted(c_dict.items(), key=lambda item: item[1]))

# Extract words and counts
words = list(c_dict.keys())
counts = list(c_dict.values())

# Plot the bar chart
plt.figure(figsize=(10, 6))
plt.bar(words, counts, color='blue')
plt.xlabel('Words')
plt.ylabel('Count')
plt.title('Word Count Distribution')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.show()

c_dict

In [75]:
total_words

8802

In [76]:
tokenizer.word_index

{'the': 1,
 'i': 2,
 'to': 3,
 'and': 4,
 'a': 5,
 'of': 6,
 'that': 7,
 'it': 8,
 'in': 9,
 'you': 10,
 'is': 11,
 'my': 12,
 'was': 13,
 'for': 14,
 'so': 15,
 'this': 16,
 'me': 17,
 'have': 18,
 'on': 19,
 'but': 20,
 'not': 21,
 'be': 22,
 'he': 23,
 'with': 24,
 'im': 25,
 'we': 26,
 'all': 27,
 'what': 28,
 'are': 29,
 'its': 30,
 'just': 31,
 'they': 32,
 'like': 33,
 'one': 34,
 'at': 35,
 'as': 36,
 'do': 37,
 'can': 38,
 'there': 39,
 'or': 40,
 'up': 41,
 'about': 42,
 'will': 43,
 'when': 44,
 'out': 45,
 'no': 46,
 'if': 47,
 'had': 48,
 'get': 49,
 'now': 50,
 'think': 51,
 'really': 52,
 'time': 53,
 'your': 54,
 'more': 55,
 'his': 56,
 'from': 57,
 'know': 58,
 'good': 59,
 'she': 60,
 'well': 61,
 'go': 62,
 'were': 63,
 'would': 64,
 'her': 65,
 'how': 66,
 'then': 67,
 'people': 68,
 'am': 69,
 'an': 70,
 'them': 71,
 'got': 72,
 'some': 73,
 'day': 74,
 'much': 75,
 'too': 76,
 'love': 77,
 'him': 78,
 'see': 79,
 'been': 80,
 'going': 81,
 'could': 82,
 'by': 83,

In [77]:
n_gram_list = []
n_gram_length = 3

for line in sentence_list:
    token_list = tokenizer.texts_to_sequences([line])[0]

    for n in range(2, n_gram_length):
        n_grams = ngrams(token_list, n)
        n_gram_list.extend(np.asarray([*n_grams]))

# Padding
n_gram_list = np.array(pad_sequences(
    n_gram_list,
    maxlen=n_gram_length,
    padding='pre'
))

In [78]:
X = n_gram_list[:, :-1]
y = n_gram_list[:, -1]

y = to_categorical(y, num_classes=total_words)

# Build and Train Model

In [81]:
from keras.src.layers import Embedding, GRU, Dense, LSTM
from keras import Sequential
import keras

model = Sequential()
model.add(Embedding(total_words, 128, input_length=n_gram_length-1))

model.add(LSTM(256))

model.add(Dense(total_words, activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 2, 128)            1126656   
                                                                 
 lstm_4 (LSTM)               (None, 256)               394240    
                                                                 
 dense_5 (Dense)             (None, 8802)              2262114   
                                                                 
Total params: 3783010 (14.43 MB)
Trainable params: 3783010 (14.43 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [82]:
from datetime import datetime

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

checkpoint_path = "checkpoints/" + datetime.now().strftime("%Y%m%d-%H%M%S") + "/" + "model_checkpoint_{epoch:02d}.h5"
checkpoint_callback = keras.callbacks.ModelCheckpoint(checkpoint_path, save_freq=5000, verbose=1)

model.fit(X, y,
          epochs=100, verbose=1,
          callbacks=[tensorboard_callback, checkpoint_callback])

Epoch 1/100
3131/3131 [==============================] - 64s 20ms/step - loss: 6.5341 - accuracy: 0.0555
Epoch 2/100
1866/3131 [================>.............] - ETA: 24s - loss: 5.9126 - accuracy: 0.1015
Epoch 2: saving model to checkpoints/20231129-175920/model_checkpoint_02.h5
3131/3131 [==============================] - 63s 20ms/step - loss: 5.8808 - accuracy: 0.1048
Epoch 3/100
3131/3131 [==============================] - 62s 20ms/step - loss: 5.5661 - accuracy: 0.1238
Epoch 4/100
 606/3131 [====>.........................] - ETA: 45s - loss: 5.2698 - accuracy: 0.1344
Epoch 4: saving model to checkpoints/20231129-175920/model_checkpoint_04.h5
3131/3131 [==============================] - 62s 20ms/step - loss: 5.3395 - accuracy: 0.1383
Epoch 5/100
2475/3131 [======================>.......] - ETA: 13s - loss: 5.1166 - accuracy: 0.1500
Epoch 5: saving model to checkpoints/20231129-175920/model_checkpoint_05.h5
3131/3131 [==============================] - 63s 20ms/step - loss: 5.1369 - 

In [1]:
import pickle

model.save("models/model_{0}.h5".format(datetime.now()).replace(" ", "_"), save_format='h5')

with open("models/tokenizer_{0}.pickle".format(datetime.now()).replace(" ", "_"), 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file, protocol=pickle.HIGHEST_PROTOCOL)

NameError: name 'model' is not defined

In [ ]:
seed_text = "Hello there, do you i am"
next_words = 1

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list],
        maxlen=2,
        padding='pre'
    )

    predictions = model.predict(token_list)
    pred_word = tokenizer.index_word[np.argmax(predictions)]
    seed_text += " " + pred_word

print("Next predicted words: ", seed_text)

In [11]:
w_l = ["hello", "word"]
w_r = ["hello", "word"]


def f():
    for word in w_r:
        
        if word in w_l:
            continue
        else:
            return True
    return False

f()

False